# Step 9: Generate contribution tables for each legislator.

In [1]:
#load everything
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from tqdm import tqdm

from operator import attrgetter

dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)

Populating the interactive namespace from numpy and matplotlib


In [2]:
legis = pd.read_sql_table('sponsor_list_det',engine)

In [3]:
#make a table for each legislator with contribution totals
for leg in tqdm(range(0,len(legis.sponsor))): #should be fast
    #first get row in legis
    legis_ind = legis[legis.index == leg]
    tableid = 'legis'+str(leg) #table id for each legislator
    #get contributions to the legislator (not aggregated)
    ind_legis_contrib = contributions.pull_contributions(legis_ind,legis,engine)
    legis_names.append(legis_ind.qsponsor.iloc[0])
    #aggregate contributions to get sum of total funding from each unique contributor.
    u_reg = np.unique(ind_legis_contrib.registrantid) #unique contributor id's.
    reg_dict = {'rid':list(), 'name':list(), 'amount':list()}
    for rid in u_reg:
        matches = ind_legis_contrib[ind_legis_contrib.registrantid == rid]
        reg_dict['rid'].append(rid)
        reg_dict['name'].append(matches.registrantname.iloc[0])
        reg_dict['amount'].append(np.sum(matches.amount))
    towrite = pd.DataFrame.from_dict(reg_dict)
    #write out the contributions from each unique contributor to a table for a given legislator.
    sqlCommands.write_to_database(dbname,tableid,towrite,engine)

100%|██████████| 389/389 [08:02<00:00,  1.15s/it]


This ends the preparation of databases for the Legislatr app to use.